# Achilles

Author: Moshe Silverstein <br/>
Date: 8-17 <br/>
Data Source: http://portals.broadinstitute.org/achilles/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import my_functions as mf
%matplotlib inline

In [2]:
importlib.reload(mf)

<module 'my_functions' from '/Users/moshesilverstein/Documents/Harmonizome/Achilles/my_functions.py'>

# Load Data

In [16]:
matrix = pd.read_csv('Input/Achilles_QC_v2.4.3.rnai.gct', sep='\t', skiprows=2)

In [17]:
matrix.head()

,Name,Description,22RV1_PROSTATE,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,786O_KIDNEY,A1207_CENTRAL_NERVOUS_SYSTEM,A172_CENTRAL_NERVOUS_SYSTEM,A204_SOFT_TISSUE,A2058_SKIN,A549_LUNG,...,TT_OESOPHAGUS,TYKNU_OVARY,U178_CENTRAL_NERVOUS_SYSTEM,U251MG_CENTRAL_NERVOUS_SYSTEM,U343_CENTRAL_NERVOUS_SYSTEM,U87MG_CENTRAL_NERVOUS_SYSTEM,UOK101_KIDNEY,VCAP_PROSTATE,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR7530_BREAST
0,GCCCACCAAAGCAAGAAACTT_61E3.4,61E3.4,-1.026155,-2.393699,-0.991135,-1.866095,-2.661319,-1.626864,1.200009,-0.162825,...,-2.066177,-0.590565,-2.387387,-1.503449,-3.352456,-0.873772,-1.554755,-1.585234,-1.627780,-1.145502
1,CAGTGACAGAAGCAGCCATAT_A1BG,A1BG,-1.621685,-1.704836,-2.818621,-1.779226,-1.844484,-2.262645,-2.723932,-2.630040,...,-3.061138,-2.665076,-1.611657,-2.642140,-1.470433,-1.223504,0.035213,-2.618386,-2.703610,-2.600708
2,CCGCCTGTGCTGATGCACCAT_A1BG,A1BG,-1.439204,-1.277287,-2.191798,-1.111282,-0.876551,0.017669,-2.646508,-1.788262,...,-0.426853,-2.829203,-0.639940,-2.060242,-0.710171,-1.428283,-1.541270,-3.182819,-1.641099,-2.069618
3,CTCTTCGAGCTGCACAACATT_A1BG,A1BG,-1.345253,-1.100449,-2.964029,-2.632790,-2.231610,-1.195225,-1.751131,-1.773799,...,-1.600203,4.345251,-0.256374,-2.461035,-1.005150,-2.055229,-1.973568,1.495429,-2.564715,-2.670118
4,GAGTCCGAATCACTGCTGAAA_A1BG,A1BG,-0.792654,-0.337205,0.277514,-0.666183,-0.833114,-0.807820,-0.442167,0.619277,...,-0.883189,-1.983697,-1.732623,0.063039,-0.162693,-2.012714,-0.735565,-0.438505,-0.786879,-0.249362


In [18]:
matrix.shape

(56903, 218)

# Load Sample Meta Data

In [10]:
sample_meta = pd.read_csv('Input/Achilles_v2.4_SampleInfo_small.txt', sep='\t')

In [11]:
sample_meta.head()

,Name,Type,Subtype,Published,Data_status,Fingerprint,QC_notes,Mean.75th.percentile.distribution.score,Conditions,Passage_number,...,KRAS,PIK3CA,BRAF,NRAS,PTEN,APC,CTNNB1,EGFR,Mut_source,GenomicDNA_isolation_date
0,22RV1_PROSTATE,Prostate,Carcinoma,NaN,"Array, Seq",match SNP6.0 ref,"Array failed QC, remove",3.862775,Adherent,8.0,...,WT,PIK3CA_Q546R,BRAF_L597R,WT,WT,WT,WT,WT,"Sanger; CCLE Oncomap, hc(BRAF)",20090812.0
1,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Leukemia,B-ALL,NaN,Seq,match SNP6.0 ref,NaN,4.180375,Suspension,8.0,...,WT,WT,WT,NRAS_G12D,WT,WT,WT,WT,"Sanger; CCLE Oncomap, hc",20110804.0
2,786O_KIDNEY,Renal Cell Carcinoma,Adenocarcinoma,Cheung et al.,"Array, Seq",match SNP6.0 ref,NaN,4.551950,Adherent,8.0,...,WT,WT,WT,WT,PTEN_Q149*,WT,WT,WT,"Sanger; CCLE Oncomap, hc",20100511.0
3,A1207_CENTRAL_NERVOUS_SYSTEM,GBM,NaN,NaN,Seq,no reference,NaN,4.381733,Adherent,9.0,...,NA/WT,NA/WT,BRAF_V600E,NA/WT,NA/WT,NA/WT,NA/WT,NA/WT,Achilles Oncomap,20120509.0
4,A172_CENTRAL_NERVOUS_SYSTEM,GBM,NaN,NaN,Seq,match SNP6.0 ref,NaN,4.249325,Adherent,NaN,...,WT,WT,WT,WT,PTEN_R55fs*1,WT,WT,WT,"Sanger(PTEN); CCLE Oncomap, hc",20110113.0


# Get Relevent Data

In [20]:
matrix = matrix.iloc[:,1:]

In [21]:
matrix.rename(columns={'Description':'Gene'}, inplace=True)

In [22]:
matrix.set_index('Gene', inplace=True)

In [34]:
lst = []

for col in matrix.columns:
    lst.append(col.split('_')[0])
    
matrix.columns = lst

In [35]:
matrix.head()

,22RV1,697,786O,A1207,A172,A204,A2058,A549,A673,ACHN,...,TT,TYKNU,U178,U251MG,U343,U87MG,UOK101,VCAP,YKG1,ZR7530
Gene,,,,,,,,,,,,,,,,,,,,,
61E3.4,-1.026155,-2.393699,-0.991135,-1.866095,-2.661319,-1.626864,1.200009,-0.162825,-1.383299,-2.619783,...,-2.066177,-0.590565,-2.387387,-1.503449,-3.352456,-0.873772,-1.554755,-1.585234,-1.627780,-1.145502
A1BG,-1.621685,-1.704836,-2.818621,-1.779226,-1.844484,-2.262645,-2.723932,-2.630040,-1.359432,-2.071313,...,-3.061138,-2.665076,-1.611657,-2.642140,-1.470433,-1.223504,0.035213,-2.618386,-2.703610,-2.600708
A1BG,-1.439204,-1.277287,-2.191798,-1.111282,-0.876551,0.017669,-2.646508,-1.788262,-1.399407,-0.752848,...,-0.426853,-2.829203,-0.639940,-2.060242,-0.710171,-1.428283,-1.541270,-3.182819,-1.641099,-2.069618
A1BG,-1.345253,-1.100449,-2.964029,-2.632790,-2.231610,-1.195225,-1.751131,-1.773799,0.655036,-1.913486,...,-1.600203,4.345251,-0.256374,-2.461035,-1.005150,-2.055229,-1.973568,1.495429,-2.564715,-2.670118
A1BG,-0.792654,-0.337205,0.277514,-0.666183,-0.833114,-0.807820,-0.442167,0.619277,-1.270333,-1.317376,...,-0.883189,-1.983697,-1.732623,0.063039,-0.162693,-2.012714,-0.735565,-0.438505,-0.786879,-0.249362


# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [36]:
mf.mapgenesymbols(matrix)

In [37]:
matrix.shape

(53503, 216)

# Merge Duplicate Genes By Rows

In [38]:
matrix = mf.merge(matrix, 'row', 'mean')

In [39]:
matrix.shape

(11980, 216)

# Save Unfiltered Matrix To File

In [40]:
filename = 'Output/achilles_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Drop Any Genes That Have Zero Expression Across 95% Of The Samples

In [41]:
matrix.replace(0, np.nan, inplace=True)

In [42]:
matrix.dropna(thresh=(0.05*matrix.shape[1]), axis=0, inplace=True)

In [43]:
matrix.replace(np.nan, 0, inplace=True)

In [44]:
matrix.shape

(11980, 216)

# Nomalize Matrix (z-score the rows)

In [46]:
normalized_matrix = matrix.copy()

In [47]:
mf.zscore(normalized_matrix, 'row')

In [48]:
normalized_matrix.head()

,22RV1,697,786O,A1207,A172,A204,A2058,A549,A673,ACHN,...,TT,TYKNU,U178,U251MG,U343,U87MG,UOK101,VCAP,YKG1,ZR7530
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.150637,0.577257,-1.217437,-0.391899,-0.170805,0.671318,-1.144494,-0.054194,1.149890,-0.318264,...,-0.272460,1.282095,0.675385,-0.890741,1.163958,-0.682283,0.689297,0.399547,-1.217091,-1.158769
A1CF,-0.647737,0.805576,-0.357445,1.395020,-0.127047,-0.009082,-1.713388,-1.012017,-1.480936,-0.683454,...,-0.745686,0.905955,-1.030863,0.362845,1.435050,0.174299,-0.911848,-0.485953,1.654512,-0.343983
A2M,-0.439085,0.071545,0.442089,-0.433158,0.592635,1.093204,-0.376625,0.243576,0.658250,1.053299,...,1.123958,-0.595001,-0.542952,0.686485,0.083591,-2.257578,-0.465275,1.159367,0.810057,2.559985
A2ML1,0.414332,0.323890,-0.568790,1.297904,-0.187955,2.055259,0.013900,1.165648,-0.151622,-0.599228,...,-0.781210,1.896407,-0.673152,-0.430485,0.957669,-0.963601,0.170557,2.042646,-1.034170,0.832668
A4GALT,0.310171,-1.507275,0.215904,1.631897,0.720867,-0.694243,0.858454,-0.406138,-0.123815,-0.148920,...,0.108907,-0.887332,-1.151974,0.375489,0.793452,0.749814,0.544340,0.730073,-0.554642,-0.073100


# Save Filtered Matrix

In [52]:
filename = 'Output/achilles_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [53]:
tertiary_matrix = mf.createTertiaryMarix(normalized_matrix)

In [54]:
tertiary_matrix.head()

,22RV1,697,786O,A1207,A172,A204,A2058,A549,A673,ACHN,...,TT,TYKNU,U178,U251MG,U343,U87MG,UOK101,VCAP,YKG1,ZR7530
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0
A2ML1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
A4GALT,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Save Teriary Matrix

In [55]:
filename = 'Output/achilles_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

#### Path to output files

In [58]:
path = '/Users/moshesilverstein/Documents/Harmonizome/Achilles/Output/'

# Create Up Gene Set Library

In [59]:
name = 'achilles_gene_up_set'

In [60]:
mf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [61]:
name = 'achilles_gene_down_set'

In [62]:
mf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [63]:
name = 'achilles_attribute_up_set'

In [64]:
mf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [65]:
name = 'achilles_attribute_down_set'

In [66]:
mf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Gene Similarity Matrix

In [67]:
gene_similarity_matix = mf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [68]:
gene_similarity_matix.head()

Gene,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AACS,AADAC,AADAT,AAK1,...,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN9,ZSWIM2,ZWILCH,ZYG11A,ZYG11B,ZYX,ZZEF1
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.159641,-0.062173,0.077849,-0.136589,-0.140795,0.056023,-0.001209,0.141038,0.148191,...,-0.224909,0.053214,0.018885,-0.069482,0.056352,-0.074837,0.129067,0.129067,0.028711,-0.013245
A1CF,-0.159641,1.000000,0.012856,0.024818,0.018352,0.134036,-0.052131,0.078772,-0.021134,0.132006,...,0.055129,0.015786,0.057882,-0.095813,0.011158,0.000556,0.037792,0.037792,-0.037190,0.196676
A2M,-0.062173,0.012856,1.000000,0.053229,-0.177169,-0.143766,-0.010045,0.098668,0.210701,-0.030437,...,0.009504,0.095373,-0.099673,-0.057722,-0.055443,0.010525,0.057545,0.057545,0.086122,-0.041541
A2ML1,0.077849,0.024818,0.053229,1.000000,-0.150917,-0.045069,-0.053243,0.162486,0.082857,-0.001906,...,0.107988,0.052065,-0.039779,0.113189,-0.101758,-0.061157,0.400307,0.400307,0.134098,0.052520
A4GALT,-0.136589,0.018352,-0.177169,-0.150917,1.000000,0.083390,-0.042429,0.078808,-0.221758,-0.169829,...,0.117682,-0.153100,0.101428,-0.176211,-0.104625,0.102026,-0.183212,-0.183212,0.091538,-0.150938


# Save Gene Similarity Matrix 

In [69]:
filename = 'Output/achilles_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute Similarity matrix

In [70]:
attribute_similarity_matix = mf.createSimilarityMatrix(normalized_matrix.T, 'cosine')

In [71]:
attribute_similarity_matix.head()

,22RV1,697,786O,A1207,A172,A204,A2058,A549,A673,ACHN,...,TT,TYKNU,U178,U251MG,U343,U87MG,UOK101,VCAP,YKG1,ZR7530
22RV1,1.000000,0.195841,-0.106514,-0.155068,0.011556,0.029933,0.018891,-0.009258,0.214370,0.035980,...,0.028026,-0.135054,0.116879,-0.040206,-0.067898,-0.054126,-0.074319,0.030735,-0.151858,0.063723
697,0.195841,1.000000,-0.088186,-0.108262,0.037689,0.077473,0.013952,-0.030635,0.247887,0.059063,...,0.010117,-0.024512,0.116777,-0.043156,0.010541,-0.030915,-0.057771,-0.014946,-0.150516,-0.102823
786O,-0.106514,-0.088186,1.000000,0.120428,0.034014,-0.004784,-0.075739,0.069607,-0.129670,0.098488,...,-0.087532,-0.014233,-0.065406,0.045354,0.078564,0.028846,0.214275,-0.065147,-0.018074,-0.027223
A1207,-0.155068,-0.108262,0.120428,1.000000,0.101372,0.140950,-0.199324,0.051486,-0.071126,0.101371,...,-0.133940,0.144375,0.054570,0.005540,0.328747,0.109269,0.140073,-0.023680,0.051034,-0.161659
A172,0.011556,0.037689,0.034014,0.101372,1.000000,0.113211,0.043932,0.071927,0.058730,0.164967,...,-0.042303,-0.064275,0.148251,0.156010,0.156517,0.192919,0.113034,-0.117892,0.114698,-0.128303


# Save Attribute Similarity Matrix

In [72]:
filename = 'Output/achilles_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [73]:
gene_list = mf.createGeneList(normalized_matrix)

In [74]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A4GALT,53947


In [75]:
gene_list.shape

(11980, 2)

# Save Gene List

In [76]:
filename = 'Output/achilles_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Attribute List 

In [77]:
attribute_list = mf.createAttributeList(normalized_matrix)

In [78]:
attribute_list.head()

,Attributes
0,22RV1
1,697
2,786O
3,A1207
4,A172


In [79]:
attribute_list.shape

(216, 1)

# Save Attribute List

In [80]:
filename = 'Output/achilles_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Gene-Attribute Edge List

In [81]:
path = '/Users/moshesilverstein/Documents/Harmonizome/Achilles/Output/'

In [82]:
name = 'achilles_gene_attribute_edge_list'

In [83]:
mf.createGeneAttributeEdgeList(tertiary_matrix, gene_list, path, name)

Progeres: 100%  216 Out of 216   

 The number of statisticaly relevent gene-attribute associations is: 517878
